# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("weather_data.csv", encoding="utf-8")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,saint-philippe,37,RE,1594961669,93,-21.36,55.77,20.00,7.20
1,busselton,38,AU,1594961670,58,-33.65,115.33,13.89,7.92
2,podgornoye,27,RU,1594961670,59,51.73,39.15,18.00,2.00
3,castro,44,BR,1594961469,86,-24.79,-50.01,13.89,2.53
4,djambala,99,CG,1594961672,90,-2.54,14.75,19.16,1.93
...,...,...,...,...,...,...,...,...,...
554,alappuzha,100,IN,1594962159,82,9.49,76.33,26.80,3.57
555,llanes,65,ES,1594962160,94,43.42,-4.75,17.78,0.45
556,strezhevoy,0,RU,1594962161,52,60.73,77.59,19.00,3.00
557,morondava,47,MG,1594962142,50,-20.28,44.28,20.84,8.55


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# CONFIGURE GMAPS
gmaps.configure(api_key=g_key)

# STORE LAT AND LONG IN LOCATIONS VARIABLE
locations = weather_data[["Lat", "Lng"]]

# STORE HUMIDITY IN HUMIDITY VARIABLE
humidity = weather_data["Humidity"]

# PLOT THE HEAT MAP
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# HEAT LAYER
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# ADD LAYER
fig.add_layer(heat_layer)

# DISPLAY
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather_data = pd.read_csv("weather_data.csv", encoding="utf-8")
weather_data

# NARROW DOWN
narrowed = weather_data

# A max temperature lower than 80 degrees but higher than 70.-- CONVERTED TO CELCIUS
narrowed = narrowed.loc[(narrowed["Max Temp"] < 26) & (narrowed["Max Temp"] > 21)]

# Wind speed < 10 mph.
narrowed = narrowed.loc[narrowed["Wind Speed"] < 10]

# Zero cloudiness.
narrowed = narrowed.loc[narrowed["Cloudiness"] == 0]

# Drop any rows with null values
narrowed = narrowed.dropna()

narrowed

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
123,pouebo,0,NC,1594961775,51,-20.40,164.57,21.71,2.30
137,ust-kalmanka,0,RU,1594961788,59,52.12,83.31,25.88,5.47
194,eugene,0,US,1594961772,63,44.05,-123.09,21.67,0.85
276,jalu,0,LY,1594961911,45,29.03,21.55,25.45,3.35
282,namibe,0,AO,1594961754,29,-15.20,12.15,21.85,5.43
365,jaru,0,BR,1594961991,52,-10.44,-62.47,21.56,0.86
377,tabriz,0,IR,1594961891,36,38.08,46.29,24.00,5.10
382,roviai,0,GR,1594962007,70,38.82,23.23,21.11,1.22
430,hervey bay,0,AU,1594962048,43,-25.30,152.85,21.11,2.68
438,awjilah,0,LY,1594962056,45,29.11,21.29,25.42,3.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create a hotel_df
hotel = narrowed.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel["Hotel Name"] = ""

# Display the result
hotel

,City,Country,Lat,Lng,Hotel Name
123,pouebo,NC,-20.40,164.57,
137,ust-kalmanka,RU,52.12,83.31,
194,eugene,US,44.05,-123.09,
276,jalu,LY,29.03,21.55,
282,namibe,AO,-15.20,12.15,
365,jaru,BR,-10.44,-62.47,
377,tabriz,IR,38.08,46.29,
382,roviai,GR,38.82,23.23,
430,hervey bay,AU,-25.30,152.85,
438,awjilah,LY,29.11,21.29,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel.iterrows():
    # CITY NAME FROM LAT AND LONG
    latitude = row["Lat"]
    longitude = row["Lng"]
    city = row["City"]
    
    # ADD KEYWORD PARAMS
    params["location"] = f"{latitude},{longitude}"

    # URL AND API REQUEST
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # RESULTS
    results = response['results']
    
    # SAVE HOTEL TO DF
    try:
        print(f"{city}'s closest hotel is {results[0]['name']}.")
        hotel.loc[index, "Hotel Name"] = results[0]['name']

    # SHOW MISSING IF NOTHING TO SHOW
    except (KeyError, IndexError):
        print("Missing ... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------END-------")

Retrieving Results for Index 123: pouebo.
Missing ... skipping.
------------
Retrieving Results for Index 137: ust-kalmanka.
Missing ... skipping.
------------
Retrieving Results for Index 194: eugene.
eugene's closest hotel is Hyatt Place Eugene / Oakway Center.
------------
Retrieving Results for Index 276: jalu.
jalu's closest hotel is Jalu Hotel.
------------
Retrieving Results for Index 282: namibe.
namibe's closest hotel is Hotel ibis Styles Iu Namibe.
------------
Retrieving Results for Index 365: jaru.
jaru's closest hotel is Hotel Pluma.
------------
Retrieving Results for Index 377: tabriz.
tabriz's closest hotel is Sina Hotel.
------------
Retrieving Results for Index 382: roviai.
roviai's closest hotel is hotel Agnanti.
------------
Retrieving Results for Index 430: hervey bay.
hervey bay's closest hotel is Big4 Hervey Bay Holiday Park.
------------
Retrieving Results for Index 438: awjilah.
Missing ... skipping.
------------
Retrieving Results for Index 499: tucurui.
tucur

In [12]:
hotel = hotel.dropna()
hotel

,City,Country,Lat,Lng,Hotel Name
123,pouebo,NC,-20.40,164.57,
137,ust-kalmanka,RU,52.12,83.31,
194,eugene,US,44.05,-123.09,Hyatt Place Eugene / Oakway Center
276,jalu,LY,29.03,21.55,Jalu Hotel
282,namibe,AO,-15.20,12.15,Hotel ibis Styles Iu Namibe
365,jaru,BR,-10.44,-62.47,Hotel Pluma
377,tabriz,IR,38.08,46.29,Sina Hotel
382,roviai,GR,38.82,23.23,hotel Agnanti
430,hervey bay,AU,-25.30,152.85,Big4 Hervey Bay Holiday Park
438,awjilah,LY,29.11,21.29,


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Lat", "Lng"]]

In [16]:
# ADD MARKER TO HEAT MAP
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# ADD LAYER
fig.add_layer(markers)

# DISPLAY
fig

Figure(layout=FigureLayout(height='420px'))